## **B Trees**

In [33]:
from time import time
from random import randint

In [34]:
class Node:
    def __init__(self,leaf=False) -> None:
        self.keys = []
        self.children = []
        self.leaf = leaf


In [69]:
class BTree:
    def __init__(self, t) -> None:
        self.root = Node(True)
        self.t = t

    def insert(self,data):
        # get tree root
        root = self.root
        t = self.t
        # split root if max keys
        if len(root.keys) == 2*t - 1:
            # create new root node, set current root as child and split
            node = Node()
            self.root = node
            self.root.children.insert(0,root)
            self.splitChildren(self.root, 0)
            self.insertNonFull(self.root, data)
        else:
            self.insertNonFull(self.root, data)

    def insertNonFull(self,root,data):
        # set i to last element
        i = len(root.keys) - 1
        if root.leaf:
            # append None element to root keys
            root.keys.append(None)
            # iterate down, insert into correct position
            while i >= 0 and data < root.keys[i]:
                root.keys[i+1] = root.keys[i]
                i -= 1
            root.keys[i+1] = data
        else:
            # iterate down to find correct child list
            while i >= 0 and data < root.keys[i]:
                i -= 1
            i += 1
            # split child if full
            if len(root.children[i].keys) == ((2*self.t) - 1):
                self.splitChildren(root,i)
                if data > root.keys[i]:
                    i += 1
            self.insertNonFull(root.children[i], data)

    def splitChildren(self, root, i):
        t = self.t
        # get root children
        y = root.children[i]
        # create new root child
        z = Node(y.leaf)
        root.children.insert(i+1, z)
        # insert median child value
        root.keys.insert(i, y.keys[t-1])
        # update (split) children
        z.keys = y.keys[t:(2*t) - 1]
        y.keys = y.keys[0:t-1]

        # inherit children
        if not y.leaf:
            z.children = y.children[t:2*t]
            y.children = y.children[0:t]

    def delete(self,target,root=None):
        if root:
            # search for node
            t = self.t
            i = 0
            if i < len(root.keys) and target > root.keys[i]:
                i += 1
            # found and root is leaf
            if root.leaf:
                if i < len(root.keys) and target == root.keys[i]:
                    root.keys.pop(i)
                return
            
            # found and root is not leaf
            if i < len(root.keys) and target == root.keys[i]:
                return self.delete_internal_node(root,target,i)
            # search children that can immediate delete
            elif len(root.children[i].keys) >= t:
                self.delete(target, root.children[i])
            # search children that need restructure
            else:
                # child has left & right siblings
                if i != 0 and i + 2 < len(root.child):
                    if len(root.children[i-1].keys) >= t:
                        self.delete_sibling(root,i,i-1)
                    elif len(root.children[i+1].keys) >= t:
                        self.delete_sibling(root,i,i+1)
                    else:
                        self.delete_merge(root,i, i+1)
                # child has only right sibling
                elif i == 0:
                    if len(root.children[i+1].keys) >= t:
                        self.delete_sibling(root,i,i+1)
                    else:
                        self.delete_merge(root,i,i+1)
                # child has only left sibling
                elif i + 1 == len(root.child):
                    if len(root.children[i-1].keys) >= t:
                        self.delete_sibling(root,i,i-1)
                    else:
                        self.delete_merge(root,i,i-1)

                self.delete(target,root.child[i])
        else:
            self.delete(target,self.root)
    
    def delete_internal_node():
        pass

    # sibling becomes parent, parent replaces/adds to children
    def delete_sibling(self,root,i,j):
        center_node = root.children[i]
        if i < j:
            right_node = root.children[j]
            # add parent key to child node
            center_node.append(root.keys[i])
            # sibling key becomes parent
            root.keys[i] = right_node.keys[0]
            # siblings left-most child becomes center node's right-most child
            if len(right_node.children) > 0:
                center_node.children.append(right_node.children[0])
                right_node.children.pop(0)
            
            right_node.keys.pop(0)

        else:
            left_node = root.children[j]
            center_node.insert(0,root.keys[i-1])
            root.keys[i-1] = left_node.keys.pop()
            if len(left_node.children) > 0:
                center_node.children.insert(0, left_node.pop())

    def delete_merge():
        pass

    def delete_predecessor(self,root):
        pass

    def delete_successor(self,root):
        pass
        



    def print_tree(self,root, l=0):
        if not root:
            return root
        print(f"Depth: {l}")
        for i in root.keys:
            print(i, end=" ")
        print()
        l += 1
        if not root.leaf:
            for i in root.children:
                self.print_tree(i,l)

Insert with Different T Size

In [87]:
N = 100000

for test in range(9):

    t = 2 + test

    bTree = BTree(t)

    start = time()
    for _ in range(N):
        bTree.insert(randint(0,1000))
    end = time()

    print(f"t={t}:\t{end-start:.8f}")

#bTree.print_tree(bTree.root)

t=2:	0.40279508
t=3:	0.26967740
t=4:	0.25221086
t=5:	0.19325376
t=6:	0.20049167
t=7:	0.18239069
t=8:	0.19807553
t=9:	0.18126464
t=10:	0.18223786
